In [1]:
from glob import glob
from model import Model
from utils import Colors, get_image_tensor, xywh2xyxy, xywh2xyxy, box_iou, ap_per_class, scale_coords
from PIL import Image
import cv2
import numpy as np
from pathlib import Path
from nms import non_max_suppression

images = glob('/data/images/val/*')
labels = glob('/data/labels/val/*')

nc = 1

stats = []

iouv = np.linspace(0.5, 0.95, 10)  # iou vector for mAP@0.5:0.95
niou = iouv.size

In [28]:
def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    print('*' * 12)
    print(labels[:, 1:], detections[:, :4])
    correct = np.zeros((detections.shape[0], iouv.shape[0])).astype(bool)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    print(iou, labels[:, 1:].shape, detections[:, :4].shape)
    correct_class = labels[:, 0:1] == detections[:, 5]
    for i in range(len(iouv)):
        x = np.where((iou >= iouv[i]) & correct_class)  # IoU > threshold and classes match
        if x[0].shape[0]:
            matches = np.cat((np.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int), i] = True
    print('*' * 12)
    return np.array(correct, dtype=bool)

In [3]:
model = Model('models/320/s.tflite', 'data/tree.yaml', conf_thresh=0.001, iou_thresh=0.6)

INFO:EdgeTPUModel:Confidence threshold: 0.001
INFO:EdgeTPUModel:IOU threshold: 0.6
INFO:EdgeTPUModel:Loaded 1 classes
INFO:EdgeTPUModel:Successfully loaded /home/mendel/code/yolo-with-coral/models/320/s.tflite


In [8]:
x = [(get_image_tensor(cv2.imread(img)),img) for img in images]

In [31]:
import numpy as np

y = []
for label in labels:
    with open(label, 'r') as f:
        lines = []
        for line in f.readlines():
            lines.append(list(map(float, line.strip().split())))
        if lines:
            lines = np.array(lines)
        else:
            lines = np.ndarray((0, 5))
        y.append(lines)

In [32]:
stats = []
for ((im, shapes), paths), (label) in zip(x, y):
    print('-' * 12)
    print(paths)
    label[:, 1:] *= [320, 320, 320, 320]
    out = model.forward(im, with_nms=False)
    print('max, min, std',out.max(), out.min(), out.std())
    
    out = non_max_suppression(prediction=out, conf_thres=0.0001, iou_thres=0.6, labels=[], agnostic=True)
#     print(out[0].shape)
    for si, pred in enumerate(out):
        shape = shapes[0]
        nl, npr = label.shape[0], pred.shape[0]
        correct = np.zeros((npr, niou)).astype(bool)
        pred[:, 5] = 0
        predn = np.copy(pred)
        scale_coords(im.shape[1:], predn[:, :4], shape, shapes[1])  # native-space pred
        if npr == 0:
            if nl:
                stats.append((correct, *np.zeros((2, 0)), label[:, 0]))
            continue

        if nl:
            tbox = xywh2xyxy(label[:, 1:])
            scale_coords(im.shape[1:], tbox, shape, shapes[1])  # native-space labels
            labeln = np.concatenate((label[:, 0:1], tbox), 1)
            correct = process_batch(predn, labeln, iouv)
            print('correct', correct.sum())
        stats.append((correct, pred[:, 4], pred[:, 5], label[:, 0]))  # (correct, conf, pcls, tcls)

------------
/data/images/val/244d39a5283f82fddcc725b87f5e19bb.png
[]
max, min, std 1057.7947 0.0 31.270315
1057.7947 0.0 (1, 6300, 85)
39
[[0. 0. 0. ... 0. 0. 0.]] 0.40595192 0.0
(39, 85) 0.0001
(39, 4) (39,) 0.6
------------
/data/images/val/f6812ab37e7a884f70a2db78de603a65.png
[]
max, min, std 802.6249 0.0 30.251078
802.6249 0.0 (1, 6300, 85)
29
[[0. 0. 0. ... 0. 0. 0.]] 0.73941237 0.0
(29, 85) 0.0001
(29, 4) (29,) 0.6
------------
/data/images/val/9b4fed2bd3e3bf4244c56058b5298eff.png
[[0.         0.48064844 0.48927546 0.44320833 0.85131944]]
max, min, std 1057.7947 0.0 30.219492
1057.7947 0.0 (1, 6300, 85)
142
[[0. 0. 0. ... 0. 0. 0.]] 0.92789006 0.0
(142, 85) 0.0001
(142, 4) (142,) 0.6
************
[[ 994.73    0.   2696.65 2160.  ]] [[1753.71203613 1052.89562988 2254.77282715 1720.97644043]
 [3284.73046875 1553.9564209  3618.77124023 2110.69042969]
 [2895.01721191 1247.75244141 3340.40441895 1526.11962891]
 [ 695.91760254  468.32510376  974.28460693  969.38562012]
 [ 640.24420166

max, min, std 1183.0598 0.0 34.746426
1183.0598 0.0 (1, 6300, 85)
38
[[0. 0. 0. ... 0. 0. 0.]] 0.88439524 0.0
(38, 85) 0.0001
(38, 4) (38,) 0.6
************
[[ 911.8           0.         1102.19        218.57777778]
 [1664.37        292.62222222 1927.6         864.14222222]
 [2094.94       2160.         2330.5        2160.        ]] [[1614.52880859  830.20220947 2059.91601562 1164.24261475]
 [1642.36523438  858.03863525 2143.42602539 1247.75262451]
 [1224.81481934  579.67172241 2338.28283691 1303.42602539]
 [1447.50842285  440.48822021 2560.97644043 1442.60943604]]
[[0.         0.         0.         0.        ]
 [0.03077681 0.00466906 0.08495348 0.09657344]
 [0.         0.         0.         0.        ]] (3, 4) (4, 4)
************
correct 0
------------
/data/images/val/62a5842c48b5fa5674c776a1f6dcc1dc.png
[[0.         0.16606641 0.53332176 0.07654948 0.143375  ]
 [0.         0.48230729 0.69699537 0.10457292 0.20184259]
 [0.         0.60700651 0.08812963 0.02460677 0.06409259]]
max, mi

max, min, std 802.6249 0.0 29.774992
802.6249 0.0 (1, 6300, 85)
0
[[0. 0. 0. ... 0. 0. 0.]] 0.0 0.0
(0, 85) 0.0001
------------
/data/images/val/d1619993b83b83d12bc07b00bab20cf5.png
[[0.         0.49922917 0.89093287 0.09895313 0.17364352]
 [0.         0.10049089 0.52401389 0.16989844 0.26100926]
 [0.         0.26504297 0.0856713  0.1061276  0.17226852]
 [0.         0.49782292 0.50674306 0.14445833 0.27384722]
 [0.         0.36542448 0.50121296 0.07517708 0.13364815]]
max, min, std 862.93774 0.0 30.152815
862.93774 0.0 (1, 6300, 85)
45
[[0. 0. 0. ... 0. 0. 0.]] 0.62342614 0.0
(45, 85) 0.0001
(45, 4) (45,) 0.6
************
[[1727.05       2160.         2107.03       2160.        ]
 [  59.68        671.07555556  712.09       1673.35111111]
 [ 814.            0.         1221.53          0.        ]
 [1634.28        580.10666667 2189.         1631.68      ]
 [1258.89        828.05333333 1547.57       1341.26222222]] [[1725.87542725 1860.16003418 1837.222229   1915.83361816]
 [1725.87542725

In [33]:
stats = [np.concatenate(x, 0) for x in zip(*stats)]

In [34]:
stats

[array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([0.40610677, 0.01450381, 0.00861821, 0.73969442, 0.02900762,
        0.01429361, 0.92824399, 0.91374022, 0.8847326 , 0.73969442,
        0.66717535, 0.3190839 , 0.02963823, 0.00924881, 0.00861821,
        0.00861821, 0.89923638, 0.8847326 , 0.82671732, 0.78320593,
        0.76870209, 0.76870209, 0.73969442, 0.37709913, 0.13053431,
        0.07251906, 0.04288084, 0.02900762, 0.02900762, 0.02900762,
        0.02774643, 0.01450381, 0.01450381, 0.01450381, 0.01450381,
        0.01450381, 0.01450381, 0.01450381, 0.01450381, 0.01450381,
        0.01450381, 0.01450381, 0.01450381, 0.01450381, 0.01450381,
        0.01450381, 0.01450381,

In [18]:
if len(stats):
    tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=True, save_dir=Path('res'),  names={0: 'Danger'})
    print(tp, fp, p, r, f1, ap, ap_class)
    ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
    mp, mr, map50, map95 = p.mean(), r.mean(), ap50.mean(), ap.mean()
nt = np.bincount(stats[3].astype(int), minlength=nc)
pf = '%20s' + '%11i' * 2 + '%11.3g' * 4  # print format

print(ap50, ap)

print('precision\t', mp)
print('recall\t', mr)
print('map .50\t', map50)
print('map .95\t', map95)

[0.] [0.] [0.] [0.] [0.] [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] [0]
[0.] [0.]
precision	 0.0
recall	 0.0
map .50	 0.0
map .95	 0.0


In [9]:
ap_class

array([0])

In [10]:
# preds = []
# labels = []
# for (full_image, net_image, pad), (label) in zip(x, y):
#     pred = model.forward(net_image)
#     preds.append(pred)
#     labels.append(label)
#     print('pred',pred)
#     print('label',label)
# #     print(model.get_scaled_coords(pred[0][:,:4], full_image, pad))

In [11]:
# net_image.shape[:2], pred[0][:, :4], [2160, 3840], shapes

In [36]:
from glob import glob

run_type = 'val'

# /home/dongha/code/
path = f'/data/tree.txt'

data = glob(f'/data/images/{run_type}/*')

print(len(data))

with open(path, 'w') as f:
    # f.writelines(data)
    for line in data:
        f.write(line)
        f.write('\n')

20


PermissionError: [Errno 13] Permission denied: '/data/tree.txt'